In [1]:
import matplotlib.pyplot as plt
import os
import pandas
import torch
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm

%matplotlib inline

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"We use: {device}")

We use: cuda


### I. Datasets and Dataloaders

##### Create train, val and test dataset

In [3]:
path_chest_imagenome_customized = "/u/home/tanida/datasets/chest-imagenome-dataset-customized-full"
csv_files_dict = {dataset: os.path.join(path_chest_imagenome_customized, dataset) + ".csv" for dataset in ["train", "valid", "test"]}

In [4]:
def print_num_samples_per_class(path_to_csv_file):
    df = pandas.read_csv(path_to_csv_file)
    

In [5]:
# specify path to folder where model weights should be saved
model_save_path = "/u/home/tanida/weights/encoder"

In [6]:
df = pandas.read_csv("/u/home/tanida/datasets/chest-imagenome-dataset-customized-full/valid.csv")

In [11]:
df["bbox_name"].value_counts()

cardiac silhouette           23949
right lung                   23946
left lung                    23944
mediastinum                  23941
upper mediastinum            23919
right upper lung zone        23918
left upper lung zone         23910
right upper abdomen          23908
abdomen                      23907
left upper abdomen           23907
trachea                      23907
left cardiac silhouette      23905
left hemidiaphragm           23904
right atrium                 23904
spine                        23904
right cardiac silhouette     23901
right costophrenic angle     23900
right hemidiaphragm          23898
left costophrenic angle      23873
cavoatrial junction          23872
right cardiophrenic angle    23872
svc                          23847
left cardiophrenic angle     23841
descending aorta             23823
right clavicle               23790
left clavicle                23776
carina                       23744
aortic arch                  23665
right lower lung zon